In [3]:
# import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV


### Read in both Training CSV and Test CSV for Kaggle

In [4]:
modelnum = 'eight_lasso'

In [5]:
# read in Training csv
df = pd.read_csv('../../datasets/train.csv')
df = df.reindex(sorted(df.columns), axis=1)
df.head()

,1st Flr SF,2nd Flr SF,3Ssn Porch,Alley,Bedroom AbvGr,Bldg Type,Bsmt Cond,Bsmt Exposure,Bsmt Full Bath,Bsmt Half Bath,...,SalePrice,Screen Porch,Street,TotRms AbvGrd,Total Bsmt SF,Utilities,Wood Deck SF,Year Built,Year Remod/Add,Yr Sold
0,725,754,0,NaN,3,1Fam,TA,No,0.0,0.0,...,130500,0,Pave,6,725.0,AllPub,0,1976,2005,2010
1,913,1209,0,NaN,4,1Fam,TA,No,1.0,0.0,...,220000,0,Pave,8,913.0,AllPub,0,1996,1997,2009
2,1057,0,0,NaN,3,1Fam,TA,No,1.0,0.0,...,109000,0,Pave,5,1057.0,AllPub,0,1953,2007,2010
3,744,700,0,NaN,3,1Fam,TA,No,0.0,0.0,...,174000,0,Pave,7,384.0,AllPub,100,2006,2007,2010
4,831,614,0,NaN,3,1Fam,Gd,No,0.0,0.0,...,138500,0,Pave,6,676.0,AllPub,0,1900,1993,2010


In [6]:
# Read in Test CSV for Kaggle
df_test = pd.read_csv('../../datasets/test.csv')
df_test = df_test.reindex(sorted(df_test.columns), axis=1)
df_test.head()

,1st Flr SF,2nd Flr SF,3Ssn Porch,Alley,Bedroom AbvGr,Bldg Type,Bsmt Cond,Bsmt Exposure,Bsmt Full Bath,Bsmt Half Bath,...,Sale Type,Screen Porch,Street,TotRms AbvGrd,Total Bsmt SF,Utilities,Wood Deck SF,Year Built,Year Remod/Add,Yr Sold
0,908,1020,0,Grvl,4,2fmCon,TA,No,0,0,...,WD,0,Pave,9,1020,AllPub,0,1910,1950,2006
1,1967,0,0,NaN,6,Duplex,TA,No,0,0,...,WD,0,Pave,10,1967,AllPub,170,1977,1977,2006
2,664,832,0,NaN,3,1Fam,Gd,Av,1,0,...,New,0,Pave,7,654,AllPub,100,2006,2006,2006
3,968,0,0,NaN,2,1Fam,TA,No,0,0,...,WD,0,Pave,5,968,AllPub,0,1923,2006,2007
4,1394,0,0,NaN,3,1Fam,TA,No,1,0,...,WD,185,Pave,6,1394,AllPub,0,1963,1963,2009


### Useful for EDA

In [7]:
# Run to see all columns in df
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [8]:
# Convert column names to lowercase
df.columns = map(str.lower,df.columns)
# Replace ' ' with '_' in column names
df.columns = df.columns.str.replace(' ','_')
df.head()

,1st_flr_sf,2nd_flr_sf,3ssn_porch,alley,bedroom_abvgr,bldg_type,bsmt_cond,bsmt_exposure,bsmt_full_bath,bsmt_half_bath,bsmt_qual,bsmt_unf_sf,bsmtfin_sf_1,bsmtfin_sf_2,bsmtfin_type_1,bsmtfin_type_2,central_air,condition_1,condition_2,electrical,enclosed_porch,exter_cond,exter_qual,exterior_1st,exterior_2nd,fence,fireplace_qu,fireplaces,foundation,full_bath,functional,garage_area,garage_cars,garage_cond,garage_finish,garage_qual,garage_type,garage_yr_blt,gr_liv_area,half_bath,heating,heating_qc,house_style,id,kitchen_abvgr,kitchen_qual,land_contour,land_slope,lot_area,lot_config,lot_frontage,lot_shape,low_qual_fin_sf,ms_subclass,ms_zoning,mas_vnr_area,mas_vnr_type,misc_feature,misc_val,mo_sold,neighborhood,open_porch_sf,overall_cond,overall_qual,pid,paved_drive,pool_area,pool_qc,roof_matl,roof_style,sale_type,saleprice,screen_porch,street,totrms_abvgrd,total_bsmt_sf,utilities,wood_deck_sf,year_built,year_remod/add,yr_sold
0,725,754,0,NaN,3,1Fam,TA,No,0.0,0.0,TA,192.0,533.0,0.0,GLQ,Unf,Y,RRAe,Norm,SBrkr,0,TA,Gd,HdBoard,Plywood,NaN,NaN,0,CBlock,2,Typ,475.0,2.0,TA,RFn,TA,Attchd,1976.0,1479,1,GasA,Ex,2Story,109,1,Gd,Lvl,Gtl,13517,CulDSac,NaN,IR1,0,60,RL,289.0,BrkFace,NaN,0,3,Sawyer,44,8,6,533352170,Y,0,NaN,CompShg,Gable,WD,130500,0,Pave,6,725.0,AllPub,0,1976,2005,2010
1,913,1209,0,NaN,4,1Fam,TA,No,1.0,0.0,Gd,276.0,637.0,0.0,GLQ,Unf,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,TA,1,PConc,2,Typ,559.0,2.0,TA,RFn,TA,Attchd,1997.0,2122,1,GasA,Ex,2Story,544,1,Gd,Lvl,Gtl,11492,CulDSac,43.0,IR1,0,60,RL,132.0,BrkFace,NaN,0,4,SawyerW,74,5,7,531379050,Y,0,NaN,CompShg,Gable,WD,220000,0,Pave,8,913.0,AllPub,0,1996,1997,2009
2,1057,0,0,NaN,3,1Fam,TA,No,1.0,0.0,TA,326.0,731.0,0.0,GLQ,Unf,Y,Norm,Norm,SBrkr,0,Gd,TA,VinylSd,VinylSd,NaN,NaN,0,CBlock,1,Typ,246.0,1.0,TA,Unf,TA,Detchd,1953.0,1057,0,GasA,TA,1Story,153,1,Gd,Lvl,Gtl,7922,Inside,68.0,Reg,0,20,RL,0.0,None,NaN,0,1,NAmes,52,7,5,535304180,Y,0,NaN,CompShg,Gable,WD,109000,0,Pave,5,1057.0,AllPub,0,1953,2007,2010
3,744,700,0,NaN,3,1Fam,TA,No,0.0,0.0,Gd,384.0,0.0,0.0,Unf,Unf,Y,Norm,Norm,SBrkr,0,TA,TA,VinylSd,VinylSd,NaN,NaN,0,PConc,2,Typ,400.0,2.0,TA,Fin,TA,BuiltIn,2007.0,1444,1,GasA,Gd,2Story,318,1,TA,Lvl,Gtl,9802,Inside,73.0,Reg,0,60,RL,0.0,None,NaN,0,4,Timber,0,5,5,916386060,Y,0,NaN,CompShg,Gable,WD,174000,0,Pave,7,384.0,AllPub,100,2006,2007,2010
4,831,614,0,NaN,3,1Fam,Gd,No,0.0,0.0,Fa,676.0,0.0,0.0,Unf,Unf,Y,Norm,Norm,SBrkr,0,TA,TA,Wd Sdng,Plywood,NaN,NaN,0,PConc,2,Typ,484.0,2.0,TA,Unf,TA,Detchd,1957.0,1445,0,GasA,TA,1.5Fin,255,1,TA,Lvl,Gtl,14235,Inside,82.0,IR1,0,50,RL,0.0,None,NaN,0,3,SawyerW,59,8,6,906425045,N,0,NaN,CompShg,Gable,WD,138500,0,Pave,6,676.0,AllPub,0,1900,1993,2010


### Identify Features to Use

In [9]:
# To Training df

# Convert column names to lowercase
df.columns = map(str.lower,df.columns)
# Replace ' ' with '_' in column names
df.columns = df.columns.str.replace(' ','_')
df.head()

df['alley'] = le.fit_transform(df['alley'])
df['bldg_type'] = le.fit_transform(df['bldg_type'])
df['central_air'] = le.fit_transform(df['central_air'])
df['condition_1'] = le.fit_transform(df['condition_1'])
df['condition_2'] = le.fit_transform(df['condition_2'])
df['electrical'] = le.fit_transform(df['electrical'])
df['exterior_1st'] = le.fit_transform(df['exterior_1st'])
df['exterior_2nd'] = le.fit_transform(df['exterior_2nd'])
df['foundation'] = le.fit_transform(df['foundation'])
df['garage_finish'] = le.fit_transform(df['garage_finish'])
df['garage_type'] = le.fit_transform(df['garage_type'])
df['heating'] = le.fit_transform(df['heating'])
df['house_style'] = le.fit_transform(df['house_style'])
df['land_contour'] = le.fit_transform(df['land_contour'])
df['lot_config'] = le.fit_transform(df['lot_config'])
df['ms_zoning'] = le.fit_transform(df['ms_zoning'])
df['mas_vnr_type'] = le.fit_transform(df['mas_vnr_type'])
df['misc_feature'] = le.fit_transform(df['misc_feature'])
df['neighborhood'] = le.fit_transform(df['neighborhood'])
df['roof_matl'] = le.fit_transform(df['roof_matl'])
df['roof_style'] = le.fit_transform(df['roof_style'])
df['sale_type'] = le.fit_transform(df['sale_type'])
df['street'] = le.fit_transform(df['street'])

df['kitchen_qual_num'] = df['kitchen_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1, 'NA':0})
df['land_slope_num'] = df['land_slope'].map({'Gtl': 2, 'Mod':1, 'Sev': 0})
df['exter_qual_num'] = df['exter_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1, 'Po':0})
df['exter_cond_num'] = df['exter_cond'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1, 'Po':0})
df['bsmt_qual_num'] = df['bsmt_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df['bsmt_cond_num'] = df['bsmt_cond'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df['bsmt_exposure_num'] = df['bsmt_exposure'].map({'Gd': 5, 'Av':4, 'Mn': 3, 'No':2, 'NA':1})
df['bsmtfin_type_1_num'] = df['bsmtfin_type_1'].map({'GLQ': 6, 'ALQ':5, 'BLQ': 4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
df['bsmtfin_type_2_num'] = df['bsmtfin_type_2'].map({'GLQ': 6, 'ALQ':5, 'BLQ': 4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
df['heating_qc_num'] = df['heating_qc'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df['electrical_num'] = df['electrical'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df['functional_num'] = df['functional'].map({'Typ': 7, 'Min2':6, 'Min1': 5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sev':1, 'Sal':0})
df['fireplace_qu_num'] = df['fireplace_qu'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df['garage_qual_num'] = df['garage_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df['garage_cond_num'] = df['garage_cond'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df['paved_drive_num'] = df['paved_drive'].map({'Y': 2, 'P':1, 'N': 0})
df['pool_qc_num'] = df['pool_qc'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'NA':1})
df['fence_num'] = df['fence'].map({'GdPrv': 5, 'MnPrv':4, 'GdWo': 3, 'MnWw':2, 'NA':1})
df['lot_shape_num'] = df['lot_shape'].map({'Reg': 3, 'IR1':2, 'IR2': 1, 'IR3':0})
df['utilities_num'] = df['utilities'].map({'AllPub': 3, 'NoSewr':2, 'NoSeWa': 1, 'ELO':0})

In [10]:
# To Test df

# Convert column names to lowercase
df_test.columns = map(str.lower,df_test.columns)
# Replace ' ' with '_' in column names
df_test.columns = df_test.columns.str.replace(' ','_')
df_test.head()

df_test['alley'] = le.fit_transform(df_test['alley'])
df_test['bldg_type'] = le.fit_transform(df_test['bldg_type'])
df_test['central_air'] = le.fit_transform(df_test['central_air'])
df_test['condition_1'] = le.fit_transform(df_test['condition_1'])
df_test['condition_2'] = le.fit_transform(df_test['condition_2'])
df_test['electrical'] = le.fit_transform(df_test['electrical'])
df_test['exterior_1st'] = le.fit_transform(df_test['exterior_1st'])
df_test['exterior_2nd'] = le.fit_transform(df_test['exterior_2nd'])
df_test['foundation'] = le.fit_transform(df_test['foundation'])
df_test['garage_finish'] = le.fit_transform(df_test['garage_finish'])
df_test['garage_type'] = le.fit_transform(df_test['garage_type'])
df_test['heating'] = le.fit_transform(df_test['heating'])
df_test['house_style'] = le.fit_transform(df_test['house_style'])
df_test['land_contour'] = le.fit_transform(df_test['land_contour'])
df_test['lot_config'] = le.fit_transform(df_test['lot_config'])
df_test['ms_zoning'] = le.fit_transform(df_test['ms_zoning'])
df_test['mas_vnr_type'] = le.fit_transform(df_test['mas_vnr_type'])
df_test['misc_feature'] = le.fit_transform(df_test['misc_feature'])
df_test['neighborhood'] = le.fit_transform(df_test['neighborhood'])
df_test['roof_matl'] = le.fit_transform(df_test['roof_matl'])
df_test['roof_style'] = le.fit_transform(df_test['roof_style'])
df_test['sale_type'] = le.fit_transform(df_test['sale_type'])
df_test['street'] = le.fit_transform(df_test['street'])

df_test['kitchen_qual_num'] = df_test['kitchen_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1, 'NA':0})
df_test['land_slope_num'] = df_test['land_slope'].map({'Gtl': 2, 'Mod':1, 'Sev': 0})
df_test['exter_qual_num'] = df_test['exter_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1, 'Po':0})
df_test['exter_cond_num'] = df_test['exter_cond'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1, 'Po':0})
df_test['bsmt_qual_num'] = df_test['bsmt_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df_test['bsmt_cond_num'] = df_test['bsmt_cond'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df_test['bsmt_exposure_num'] = df_test['bsmt_exposure'].map({'Gd': 5, 'Av':4, 'Mn': 3, 'No':2, 'NA':1})
df_test['bsmtfin_type_1_num'] = df_test['bsmtfin_type_1'].map({'GLQ': 6, 'ALQ':5, 'BLQ': 4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
df_test['bsmtfin_type_2_num'] = df_test['bsmtfin_type_2'].map({'GLQ': 6, 'ALQ':5, 'BLQ': 4, 'Rec':3, 'LwQ':2, 'Unf':1, 'NA':0})
df_test['heating_qc_num'] = df_test['heating_qc'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df_test['electrical_num'] = df_test['electrical'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df_test['functional_num'] = df_test['functional'].map({'Typ': 7, 'Min2':6, 'Min1': 5, 'Mod':4, 'Maj1':3, 'Maj2':2, 'Sev':1, 'Sal':0})
df_test['fireplace_qu_num'] = df_test['fireplace_qu'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df_test['garage_qual_num'] = df_test['garage_qual'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df_test['garage_cond_num'] = df_test['garage_cond'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'Po':1})
df_test['paved_drive_num'] = df_test['paved_drive'].map({'Y': 2, 'P':1, 'N': 0})
df_test['pool_qc_num'] = df_test['pool_qc'].map({'Ex': 5, 'Gd':4, 'TA': 3, 'Fa':2, 'NA':1})
df_test['fence_num'] = df_test['fence'].map({'GdPrv': 5, 'MnPrv':4, 'GdWo': 3, 'MnWw':2, 'NA':1})
df_test['lot_shape_num'] = df_test['lot_shape'].map({'Reg': 3, 'IR1':2, 'IR2': 1, 'IR3':0})
df_test['utilities_num'] = df_test['utilities'].map({'AllPub': 3, 'NoSewr':2, 'NoSeWa': 1, 'ELO':0})

In [11]:
# Replace NA with mean
# To Training

df['bsmt_cond_num'] = df['bsmt_cond_num'].fillna(df['bsmt_cond_num'].mean())
df['bsmt_exposure_num'] = df['bsmt_exposure_num'].fillna(df['bsmt_exposure_num'].mean())
df['bsmt_full_bath'] = df['bsmt_full_bath'].fillna(df['bsmt_full_bath'].mean())
df['bsmt_half_bath'] = df['bsmt_half_bath'].fillna(df['bsmt_half_bath'].mean())
df['bsmt_qual_num'] = df['bsmt_qual_num'].fillna(df['bsmt_qual_num'].mean())
df['bsmt_unf_sf'] = df['bsmt_unf_sf'].fillna(df['bsmt_unf_sf'].mean())
df['bsmtfin_sf_1'] = df['bsmtfin_sf_1'].fillna(df['bsmtfin_sf_1'].mean())
df['bsmtfin_sf_2'] = df['bsmtfin_sf_2'].fillna(df['bsmtfin_sf_2'].mean())
df['bsmtfin_type_1_num'] = df['bsmtfin_type_1_num'].fillna(df['bsmtfin_type_1_num'].mean())
df['bsmtfin_type_2_num'] = df['bsmtfin_type_2_num'].fillna(df['bsmtfin_type_2_num'].mean())
df['garage_area'] = df['garage_area'].fillna(df['garage_area'].mean())
df['garage_cars'] = df['garage_cars'].fillna(df['garage_cars'].mean())
df['garage_cond_num'] = df['garage_cond_num'].fillna(df['garage_cond_num'].mean())
df['garage_qual_num'] = df['garage_qual_num'].fillna(df['garage_qual_num'].mean())
df['garage_yr_blt'] = df['garage_yr_blt'].fillna(df['garage_yr_blt'].mean())
df['mas_vnr_area'] = df['mas_vnr_area'].fillna(df['mas_vnr_area'].mean())
df['fireplace_qu_num'] = df['fireplace_qu_num'].fillna(df['fireplace_qu_num'].mean())
df['lot_frontage'] = df['lot_frontage'].fillna(df['lot_frontage'].mean())
df['total_bsmt_sf'] = df['total_bsmt_sf'].fillna(df['total_bsmt_sf'].mean())

# To Testing

df_test['bsmt_cond_num'] = df_test['bsmt_cond_num'].fillna(df_test['bsmt_cond_num'].mean())
df_test['bsmt_exposure_num'] = df_test['bsmt_exposure_num'].fillna(df_test['bsmt_exposure_num'].mean())
df_test['bsmt_full_bath'] = df_test['bsmt_full_bath'].fillna(df_test['bsmt_full_bath'].mean())
df_test['bsmt_half_bath'] = df_test['bsmt_half_bath'].fillna(df_test['bsmt_half_bath'].mean())
df_test['bsmt_qual_num'] = df_test['bsmt_qual_num'].fillna(df_test['bsmt_qual_num'].mean())
df_test['bsmt_unf_sf'] = df_test['bsmt_unf_sf'].fillna(df_test['bsmt_unf_sf'].mean())
df_test['bsmtfin_sf_1'] = df_test['bsmtfin_sf_1'].fillna(df_test['bsmtfin_sf_1'].mean())
df_test['bsmtfin_sf_2'] = df_test['bsmtfin_sf_2'].fillna(df_test['bsmtfin_sf_2'].mean())
df_test['bsmtfin_type_1_num'] = df_test['bsmtfin_type_1_num'].fillna(df_test['bsmtfin_type_1_num'].mean())
df_test['bsmtfin_type_2_num'] = df_test['bsmtfin_type_2_num'].fillna(df_test['bsmtfin_type_2_num'].mean())
df_test['garage_area'] = df_test['garage_area'].fillna(df_test['garage_area'].mean())
df_test['garage_cars'] = df_test['garage_cars'].fillna(df_test['garage_cars'].mean())
df_test['garage_cond_num'] = df_test['garage_cond_num'].fillna(df_test['garage_cond_num'].mean())
df_test['garage_qual_num'] = df_test['garage_qual_num'].fillna(df_test['garage_qual_num'].mean())
df_test['garage_yr_blt'] = df_test['garage_yr_blt'].fillna(df_test['garage_yr_blt'].mean())
df_test['mas_vnr_area'] = df_test['mas_vnr_area'].fillna(df_test['mas_vnr_area'].mean())
df_test['fireplace_qu_num'] = df_test['fireplace_qu_num'].fillna(df_test['fireplace_qu_num'].mean())
df_test['lot_frontage'] = df_test['lot_frontage'].fillna(df_test['lot_frontage'].mean())
df_test['total_bsmt_sf'] = df_test['total_bsmt_sf'].fillna(df_test['total_bsmt_sf'].mean())

In [12]:
df_test = df_test.reindex(sorted(df_test.columns), axis=1)
df = df.reindex(sorted(df.columns), axis=1)

### Time to Dive into the Data!!!

#### Feature Engineering Happens RIGHT Here!!

In [13]:
df.head()

,1st_flr_sf,2nd_flr_sf,3ssn_porch,alley,bedroom_abvgr,bldg_type,bsmt_cond,bsmt_cond_num,bsmt_exposure,bsmt_exposure_num,bsmt_full_bath,bsmt_half_bath,bsmt_qual,bsmt_qual_num,bsmt_unf_sf,bsmtfin_sf_1,bsmtfin_sf_2,bsmtfin_type_1,bsmtfin_type_1_num,bsmtfin_type_2,bsmtfin_type_2_num,central_air,condition_1,condition_2,electrical,electrical_num,enclosed_porch,exter_cond,exter_cond_num,exter_qual,exter_qual_num,exterior_1st,exterior_2nd,fence,fence_num,fireplace_qu,fireplace_qu_num,fireplaces,foundation,full_bath,functional,functional_num,garage_area,garage_cars,garage_cond,garage_cond_num,garage_finish,garage_qual,garage_qual_num,garage_type,garage_yr_blt,gr_liv_area,half_bath,heating,heating_qc,heating_qc_num,house_style,id,kitchen_abvgr,kitchen_qual,kitchen_qual_num,land_contour,land_slope,land_slope_num,lot_area,lot_config,lot_frontage,lot_shape,lot_shape_num,low_qual_fin_sf,mas_vnr_area,mas_vnr_type,misc_feature,misc_val,mo_sold,ms_subclass,ms_zoning,neighborhood,open_porch_sf,overall_cond,overall_qual,paved_drive,paved_drive_num,pid,pool_area,pool_qc,pool_qc_num,roof_matl,roof_style,sale_type,saleprice,screen_porch,street,total_bsmt_sf,totrms_abvgrd,utilities,utilities_num,wood_deck_sf,year_built,year_remod/add,yr_sold
0,725,754,0,2,3,0,TA,3.0,No,2.0,0.0,0.0,TA,3.0,192.0,533.0,0.0,GLQ,6.0,Unf,1.0,1,5,2,4,NaN,0,TA,3,Gd,4,6,9,NaN,NaN,NaN,3.441484,0,1,2,Typ,7,475.0,2.0,TA,3.0,1,TA,3.0,1,1976.0,1479,1,0,Ex,5,5,109,1,Gd,4,3,Gtl,2,13517,1,69.0552,IR1,2,0,289.0,1,5,0,3,60,5,22,44,8,6,Y,2,533352170,0,NaN,NaN,1,1,8,130500,0,1,725.0,6,AllPub,3,0,1976,2005,2010
1,913,1209,0,2,4,0,TA,3.0,No,2.0,1.0,0.0,Gd,4.0,276.0,637.0,0.0,GLQ,6.0,Unf,1.0,1,2,2,4,NaN,0,TA,3,Gd,4,12,12,NaN,NaN,TA,3.000000,1,2,2,Typ,7,559.0,2.0,TA,3.0,1,TA,3.0,1,1997.0,2122,1,0,Ex,5,5,544,1,Gd,4,3,Gtl,2,11492,1,43.0000,IR1,2,0,132.0,1,5,0,4,60,5,23,74,5,7,Y,2,531379050,0,NaN,NaN,1,1,8,220000,0,1,913.0,8,AllPub,3,0,1996,1997,2009
2,1057,0,0,2,3,0,TA,3.0,No,2.0,1.0,0.0,TA,3.0,326.0,731.0,0.0,GLQ,6.0,Unf,1.0,1,2,2,4,NaN,0,Gd,4,TA,3,12,12,NaN,NaN,NaN,3.441484,0,1,1,Typ,7,246.0,1.0,TA,3.0,2,TA,3.0,5,1953.0,1057,0,0,TA,3,2,153,1,Gd,4,3,Gtl,2,7922,4,68.0000,Reg,3,0,0.0,2,5,0,1,20,5,15,52,7,5,Y,2,535304180,0,NaN,NaN,1,1,8,109000,0,1,1057.0,5,AllPub,3,0,1953,2007,2010
3,744,700,0,2,3,0,TA,3.0,No,2.0,0.0,0.0,Gd,4.0,384.0,0.0,0.0,Unf,1.0,Unf,1.0,1,2,2,4,NaN,0,TA,3,TA,3,12,12,NaN,NaN,NaN,3.441484,0,2,2,Typ,7,400.0,2.0,TA,3.0,0,TA,3.0,3,2007.0,1444,1,0,Gd,4,5,318,1,TA,3,3,Gtl,2,9802,4,73.0000,Reg,3,0,0.0,2,5,0,4,60,5,26,0,5,5,Y,2,916386060,0,NaN,NaN,1,1,8,174000,0,1,384.0,7,AllPub,3,100,2006,2007,2010
4,831,614,0,2,3,0,Gd,4.0,No,2.0,0.0,0.0,Fa,2.0,676.0,0.0,0.0,Unf,1.0,Unf,1.0,1,2,2,4,NaN,0,TA,3,TA,3,13,9,NaN,NaN,NaN,3.441484,0,2,2,Typ,7,484.0,2.0,TA,3.0,2,TA,3.0,5,1957.0,1445,0,0,TA,3,0,255,1,TA,3,3,Gtl,2,14235,4,82.0000,IR1,2,0,0.0,2,5,0,3,50,5,23,59,8,6,N,0,906425045,0,NaN,NaN,1,1,8,138500,0,1,676.0,6,AllPub,3,0,1900,1993,2010


In [14]:
# to df

df['overall_qual_squared'] = (df['overall_qual'] ** 2) 
df['quality_and_age'] = df['overall_qual'] * df['year_remod/add']
df['house_total_sq'] = df['total_bsmt_sf'] + df['gr_liv_area']
df['sqrt_1st_sq'] = (df['1st_flr_sf']**2)
df['overall_qual_liv_area'] = df['overall_qual'] * df['gr_liv_area']
df['year_remod_or_built'] = df['year_remod/add'] * df['year_built']
df['kitch_qual_and_age'] = df['kitchen_qual'] * df['year_remod/add']
df['year_remod_or_built_sq'] = (df['year_remod_or_built'] ** 2)
df['exterior_cond_and_qual'] = df['exter_cond_num'] * df['exter_qual_num']

# to df_test

df_test['overall_qual_squared'] = (df_test['overall_qual'] ** 2) 
df_test['quality_and_age'] = df_test['overall_qual'] * df_test['year_remod/add']
df_test['house_total_sq'] = df_test['total_bsmt_sf'] + df_test['gr_liv_area']
df_test['sqrt_1st_sq'] = (df_test['1st_flr_sf']**2)
df_test['overall_qual_liv_area'] = df_test['overall_qual'] * df_test['gr_liv_area']
df_test['year_remod_or_built'] = df_test['year_remod/add'] * df_test['year_built']
df_test['kitch_qual_and_age'] = df_test['kitchen_qual'] * df_test['year_remod/add']
df_test['year_remod_or_built_sq'] = (df_test['year_remod_or_built'] ** 2)
df_test['exterior_cond_and_qual'] = df_test['exter_cond_num'] * df_test['exter_qual_num']

In [15]:
# Top 10
df.corr()['saleprice'].sort_values(ascending=False).head(15)

saleprice                 1.000000
overall_qual_liv_area     0.837152
overall_qual_squared      0.825539
quality_and_age           0.804740
overall_qual              0.800207
house_total_sq            0.779286
exter_qual_num            0.712146
gr_liv_area               0.697038
kitchen_qual_num          0.692336
bsmt_qual_num             0.667041
garage_area               0.650246
garage_cars               0.648197
total_bsmt_sf             0.628668
year_remod_or_built_sq    0.624009
year_remod_or_built       0.622283
Name: saleprice, dtype: float64

In [16]:
# Bottom 5
df.corr()['saleprice'].sort_values(ascending=False).tail(5)

pid              -0.255052
lot_shape_num    -0.294542
garage_type      -0.427721
garage_finish    -0.557839
electrical_num         NaN
Name: saleprice, dtype: float64

In [17]:
'-' * 160

'----------------------------------------------------------------------------------------------------------------------------------------------------------------'

#### Once Features have been decided upon, we need to Clean them and Feature Engineer them

In [18]:
features = ['overall_qual_liv_area', 'overall_qual_squared', 'quality_and_age', 'overall_qual', 'house_total_sq', 'exter_qual_num', 'gr_liv_area', 'kitchen_qual_num', 'bsmt_qual_num', 'garage_area',
           'garage_cars', 'total_bsmt_sf', 'year_remod_or_built', 'year_remod_or_built_sq', 'garage_finish', 'exterior_cond_and_qual']

X = df[features]
y = df['saleprice']

### Do all of the Cleaning and Engineering stuff to the Test.csv for Kaggle

### Once Features have been cleaned, we need to instantiate the model

In [19]:
# Train, Test, Split

X_train, X_validate, y_train, y_validate = train_test_split(X, y, random_state=42)

In [28]:
# Instantiate and scale data
sc = StandardScaler()

X_train_sc = sc.fit_transform(X_train)
X_validate_sc = sc.transform(X_validate)

In [29]:
# Set up a list of Lasso alphas to check.
l_alphas = np.logspace(-3, 0, 100)
# Cross-validate over our list of Lasso alphas.
lasso_cv = LassoCV(alphas = l_alphas)
# Fit model using best ridge alpha!
lasso_cv.fit(X_train_sc, y_train)

/Users/blackmagic/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 713268314091.9785, tolerance: 778133067.1215105
  model = cd_fast.enet_coordinate_descent_gram(
/Users/blackmagic/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 711742152328.6428, tolerance: 778133067.1215105
  model = cd_fast.enet_coordinate_descent_gram(
/Users/blackmagic/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 710390720918.1859, tolerance: 778133067.1215105
  model = cd_fast.enet_coordinate_descent_gram(
/Users/blackmagic/opt/anaconda3/lib/python3

LassoCV(alphas=array([0.001     , 0.00107227, 0.00114976, 0.00123285, 0.00132194,
       0.00141747, 0.00151991, 0.00162975, 0.00174753, 0.00187382,
       0.00200923, 0.00215443, 0.00231013, 0.00247708, 0.00265609,
       0.00284804, 0.00305386, 0.00327455, 0.00351119, 0.00376494,
       0.00403702, 0.00432876, 0.00464159, 0.00497702, 0.0053367 ,
       0.00572237, 0.00613591, 0.00657933, 0.0070548 , 0.00756463,
       0.008...
       0.09326033, 0.1       , 0.10722672, 0.1149757 , 0.12328467,
       0.13219411, 0.14174742, 0.15199111, 0.16297508, 0.17475284,
       0.18738174, 0.2009233 , 0.21544347, 0.23101297, 0.24770764,
       0.26560878, 0.28480359, 0.30538555, 0.32745492, 0.35111917,
       0.37649358, 0.40370173, 0.43287613, 0.46415888, 0.49770236,
       0.53366992, 0.57223677, 0.61359073, 0.65793322, 0.70548023,
       0.75646333, 0.81113083, 0.869749  , 0.93260335, 1.        ]))

In [31]:
# Here is the optimal value of alpha
lasso_cv.alpha_

0.001

In [34]:
# R2 Training Score, R2 Test Score
lasso_cv.score(X_train_sc, y_train), lasso_cv.score(X_validate_sc, y_validate)

(0.8124077317505859, 0.8691990572980588)

In [33]:
# Baseline Model
baseline = y.mean()
baseline

181469.70160897123